In [203]:
import pandas as pd
import base64

from src.utils import load_data_label_vector

In [204]:
rule_ids = ['942011', '942012', '942013', '942014', '942015', '942016', '942017', '942018', '942100', '942101', '942110', '942120', '942130', '942131', '942140', '942150', '942151', '942152', '942160', '942170', '942180', '942190', '942200', '942210', '942220', '942230', '942240', '942250', '942251', '942260', '942270', '942280', '942290', '942300', '942310', '942320', '942321', '942330', '942340', '942350', '942360', '942361', '942362', '942370', '942380', '942390', '942400', '942410', '942420', '942421', '942430', '942431', '942432', '942440', '942450', '942460', '942470', '942480', '942490', '942500', '942510', '942511', '942520', '942521', '942522', '942530', '942540', '942550', '942560']
print(f"Rule IDs: {rule_ids}")

Rule IDs: ['942011', '942012', '942013', '942014', '942015', '942016', '942017', '942018', '942100', '942101', '942110', '942120', '942130', '942131', '942140', '942150', '942151', '942152', '942160', '942170', '942180', '942190', '942200', '942210', '942220', '942230', '942240', '942250', '942251', '942260', '942270', '942280', '942290', '942300', '942310', '942320', '942321', '942330', '942340', '942350', '942360', '942361', '942362', '942370', '942380', '942390', '942400', '942410', '942420', '942421', '942430', '942431', '942432', '942440', '942450', '942460', '942470', '942480', '942490', '942500', '942510', '942511', '942520', '942521', '942522', '942530', '942540', '942550', '942560']


In [205]:
data = load_data_label_vector("/app/wafcraft/data/prepared/2024-04-18_14-12-51_lightblue-around/test.csv")

random_entry = data.sample(1).iloc[0]

print(f"Label: {'attack' if random_entry['label'] == 1 else 'normal'}")
print(f"Data (base64): {random_entry['data']}")
print(f"Data (unbase64): {base64.b64decode(random_entry['data'])}")
# print(f"Vector (list): {list(random_entry['vector'])}")

# # find the active rules, eg. where the vector is 1
active_rules = [rule_ids[i] for i, v in enumerate(random_entry['vector']) if v == 1]
print(f"Active rules: {len(active_rules)}")
# print active rules, each on a new line
for rule in active_rules:
    print(rule)


Label: attack
Data (base64): REVMRVRFIEZST00gYHRhYmAgV0hFUkUgYGNvbDFgIExJS0UgIiUnbCclIiBBTkQgYGNvbDNgIExJS0UgIiUnYSclIiBMSU1JVCAnIikpKSBBTkQgMzUyNj0oU0VMRUNUIENPVU5UKCopIEZST00gUkRCJEZJRUxEUyBBUyBUMSxSREIkVFlQRVMgQVMgVDIsUkRCJENPTExBVElPTlMgQVMgVDMsUkRCJEZVTkNUSU9OUyBBUyBUNCktLSc7
Data (unbase64): b'DELETE FROM `tab` WHERE `col1` LIKE "%\'l\'%" AND `col3` LIKE "%\'a\'%" LIMIT \'"))) AND 3526=(SELECT COUNT(*) FROM RDB$FIELDS AS T1,RDB$TYPES AS T2,RDB$COLLATIONS AS T3,RDB$FUNCTIONS AS T4)--\';'
Active rules: 20
942100
942150
942180
942200
942210
942260
942290
942310
942370
942380
942400
942410
942430
942431
942432
942460
942480
942510
942520
942530


In [206]:
import joblib

from src.model import create_wafamole_model
from src.modsec import init_modsec

modsec = init_modsec()
model = joblib.load(
    "/app/wafcraft/data/prepared/2024-04-07_18-15-53_brown-lot/model_adv/model_adv.joblib"
)
wafamole_model = create_wafamole_model(model, modsec, rule_ids, 4)

# weird_payload = b"SELECT `col2`, `col2`, `col2`, `col3` FROM `tab` WHERE `col1` LIKE \"%'ujsanclkmksqiopdfyvpqzizzzuq'%\" LIMIT -2078343168,-910688256;"
# calc_vec = wafamole_model.extract_features(weird_payload.decode("utf-8"))

calc_vec = wafamole_model.extract_features(
    base64.b64decode(random_entry["data"]).decode("utf-8")
)
print(calc_vec)
active_rules2 = [rule_ids[i] for i, v in enumerate(calc_vec) if v == 1]
print(f"Active rules: {len(active_rules2)}")
# print active rules, each on a new line
for rule in active_rules2:
    print(rule)
print(wafamole_model.classify(base64.b64decode(random_entry["data"]).decode("utf-8")))

[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 0 1 0 0
 0 0 0 0 0 0 1 1 0 1 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 0 0]
Active rules: 20
942100
942150
942180
942200
942210
942260
942290
942310
942370
942380
942400
942410
942430
942431
942432
942460
942480
942510
942520
942530
1.0
